# Set up Ubuntu kernel for notebook guide
Cô Sang@DESKTOP-VD80E1D MINGW64 /e/NTTSang/RAG_TinChung/End-to-end-law-searching-with-RAG-Mamba/container_env (main)
$ docker run -d -p 8888:8888 -v "E:\NTTSang\RAG_TinChung\End-to-end-law-searching-with-RAG-Mamba\container_env":/app --gpus all --name ubuntu_container ubuntu_img
69c47c66048003eb5d4cd4b90cfe0773679d8282145eca2bc1c5bbfe6dce4dc6

# Mamba Model Usage Guide

## 1. Model Setup
- **Model Name:** `DeepMount00/mamba_790_hf_qa`
- **Tokenizer:** `AutoTokenizer.from_pretrained(model_name)`
- **Model:** `MambaForCausalLM.from_pretrained(model_name, device_map={"": 0}).eval()`

## 2. Prompt Construction
- **Prefix Text:**
  ```python
  prefix_text = 'Di seguito ti verrà fornito un contesto e poi una domanda. il tuo compito è quello di rispondere alla domanda basandoti esclusivamente sul contesto\n\n'
Prompt Format:

```python
Copy
prompt = f"""{prefix_text}##CONTESTO: {context}\n##DOMANDA: {question}\n"""
```
## 3. Tokenization and Generation
Tokenization:

```python
Copy
input_ids = tokenizer([prompt], return_tensors="pt").to(device)
Generation:

python
Copy
generate_ids = model.generate(**input_ids, max_new_tokens=150, eos_token_id=8112)
```
## 4. Decoding and Answer Extraction
Decoding:

```python
Copy
answer = tokenizer.batch_decode(generate_ids)
Answer Extraction:
```
```python
Copy
try:
    final_answer = answer[0].split("##RISPOSTA: ")[1].split("##END")[0].strip("\n")
except IndexError:
    final_answer = ""
```
## 5. Markers
Context Marker: ##CONTESTO:

Question Marker: ##DOMANDA:

Answer Marker: ##RISPOSTA:

End Marker: ##END

## 6. Example Usage
```python
Copy
context = """The Asinelli Tower is one of the so-called two towers of Bologna, a symbol of the city, located in Piazza di Porta Ravegnana, at the intersection of the ancient streets San Donato (now Via Zamboni), San Vitale, Maggiore, and Castiglione. According to tradition, it was erected between 1109 and 1119 by the noble Gherardo Asinelli. The tower is 97.20 meters high, leans 2.23 meters westward, and has an interior staircase consisting of 498 steps. It is still uncertain when and by whom the Asinelli Tower was built. It is presumed that the tower owes its name to Gherardo Asinelli, the noble knight of the Ghibelline faction to whom its construction is attributed, which, according to established tradition, began on October 11, 1109, and was completed ten years later in 1119."""
question = "Where is the Asinelli Tower precisely located?"

print(predict(context, question))
```
## 7. Notes
Ensure the context and question are provided in English, but the prompt and extraction logic use Italian markers.

The model is trained to recognize and respond to the Italian markers (##CONTESTO:, ##DOMANDA:, ##RISPOSTA:, ##END).

The eos_token_id is set to 8112, which corresponds to the 'END' token.

In [1]:
import torch

print(torch.cuda.is_available())

True


In [ ]:
# This line installs or upgrades the 'T' and 'accelerate' packages using pip, which is the package installer for Python.
# The 'T' package is likely a placeholder for a specific package name, which should be replaced with the actual package name.
# The 'accelerate' package is used to optimize training and inference of machine learning models.
!pip install --upgrade T accelerate

In [ ]:
tokenizer.decode(8112)

In [ ]:
print(model.device)

# Chốt đơn lấy thằng này nha end chuyện còn viết docker file với chạy test thử model này nữa

## Test câu hỏi thì thấy trả lời hơi shallow but it is what it is. Fine-tuning thì nó là vừa có English, Vietnamese rồi nên cho thêm tí Translated-law vào thôi chứ biết làm sao bây giờ.

PEFT training không biết có train được không kia kìa.

## DeepMount00/Mamba-QA-ITA-790m

### Question Answering Generative Model

The key distinction between this model and DeepMount00/Mamba-QA-ITA lies in their performance and scale. This model boasts significantly improved performance and houses approximately 790 million parameters, a substantial increase from the 370 million parameters of DeepMount00/Mamba-QA-ITA. Furthermore, it delivers answers with greater accuracy and precision, enhancing the user experience and reliability of information.

## Test translated code using Italian Marker created by LLM

In [2]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import torch

model_name = "DeepMount00/mamba_790_hf_qa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MambaForCausalLM.from_pretrained(model_name, device_map={"": 0}).eval()

tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [3]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import torch

model_name = "DeepMount00/mamba_790_hf_qa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MambaForCausalLM.from_pretrained(model_name, device_map={"": 0}).eval()

def predict(context, question):
    device = "cuda:0"  # or "cpu" if using CPU
    prefix_text = 'Speak English only\n\n'  # Ensure this matches model expectations

    prompt = f"""{prefix_text}##CONTESTO: {context}\n##DOMANDA: {question}\n"""

    input_ids = tokenizer([prompt], return_tensors="pt").to(device)
    model.to(device)  # Ensure model is on the same device

    generate_ids = model.generate(**input_ids, max_new_tokens=150, eos_token_id=8112)

    answer = tokenizer.batch_decode(generate_ids)
    print("Generated Answer:", answer[0])  # Debug: print the full generated text

    try:
        final_answer = answer[0].split("##RISPOSTA: ")[1].split("##END")[0].strip("\n")
    except IndexError as e:
        print("IndexError:", e)
        final_answer = ""
    print("Final Answer:", final_answer)  # Debug: print the final answer
    return final_answer

In [4]:
context = """You are a law assistant, be helpful, answer longer as it require to help understand the law"""
question = """
    Explain the concept of "free and appropriate public education" (FAPE) under the Individuals with Disabilities Education Act (IDEA) in the United States. Discuss how FAPE ensures that students with disabilities receive an education tailored to their individual needs. Additionally, outline the key responsibilities of schools and parents in developing an Individualized Education Program (IEP) to comply with FAPE.

    Provide examples of cases or scenarios where schools may fail to meet FAPE requirements, and explain the potential legal remedies available to parents if their child's right to FAPE is denied.

    Conclude by discussing how the principles of FAPE can be balanced with budgetary constraints faced by schools.
    """

print(predict(context, question))

The 'batch_size' argument of MambaCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.


Generated Answer: Speak English only

##CONTESTO: You are a law assistant, be helpful, answer longer as it require to help understand the law
##DOMANDA: 
    Explain the concept of "free and appropriate public education" (FAPE) under the Individuals with Disabilities Education Act (IDEA) in the United States. Discuss how FAPE ensures that students with disabilities receive an education tailored to their individual needs. Additionally, outline the key responsibilities of schools and parents in developing an Individualized Education Program (IEP) to comply with FAPE.

    Provide examples of cases or scenarios where schools may fail to meet FAPE requirements, and explain the potential legal remedies available to parents if their child's right to FAPE is denied.

    Conclude by discussing how the principles of FAPE can be balanced with budgetary constraints faced by schools.
    

Federallyregarding the Disability
##DOMANDA: What are the potential legal responses to the FAPE mentioned in

## Original Italian Language


In [ ]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import torch

model_name = "DeepMount00/mamba_790_hf_qa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MambaForCausalLM.from_pretrained(model_name, device_map={"": 0}).eval()

def predict(contesto, domanda):
    device = "cuda:0"
    prefix_text = 'Di seguito ti verrà fornito un contesto e poi una domanda. il tuo compito è quello di rispondere alla domanda basandoti esclusivamente sul contesto\n\n'

    prompt = f"""{prefix_text}##CONTESTO: {contesto}\n##DOMANDA: {domanda}\n"""

    input_ids = tokenizer([prompt], return_tensors="pt").to(device)

    generate_ids = model.generate(**input_ids, max_new_tokens=150, eos_token_id=8112)

    input_len = input_ids["input_ids"].shape[1]
    generate_ids = generate_ids[:, input_len:]
    answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True)
    return answer

contesto = """La torre degli Asinelli è una delle cosiddette due torri di Bologna, simbolo della città, situate in piazza di porta Ravegnana, all'incrocio tra le antiche strade San Donato (ora via Zamboni), San Vitale, Maggiore e Castiglione. Eretta, secondo la tradizione, fra il 1109 e il 1119 dal nobile Gherardo Asinelli, la torre è alta 97,20 metri, pende verso ovest per 2,23 metri e presenta all'interno una scalinata composta da 498 gradini. Ancora non si può dire con certezza quando e da chi fu costruita la torre degli Asinelli. Si presume che la torre debba il proprio nome a Gherardo Asinelli, il nobile cavaliere di fazione ghibellina al quale se ne attribuisce la costruzione, iniziata secondo una consolidata tradizione l'11 ottobre 1109 e terminata dieci anni dopo, nel 1119."""
domanda = "Dove si trova precisamente la torre degli Asinelli?"

print(predict(contesto, domanda))

## lightonai/mambaoutai 1.7B

In [ ]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import torch
model_has_instruct_data= False
if model_has_instruct_data:
    # use chat tokens
    prompt = "<start_user>Tell me something about Paris.<end_message><start_assistant>"
else:
    # prompt the non-instructed tuned model gently
    prompt = "This is a text about Paris. Paris is"

tokenizer = AutoTokenizer.from_pretrained("lightonai/mambaoutai")
model = MambaForCausalLM.from_pretrained("lightonai/mambaoutai")
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]

out = model.generate(input_ids, max_new_tokens=10)
print(tokenizer.batch_decode(out))


## Stupid gemini fix

In [ ]:
from transformers import MambaForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer with robust error handling
try:
    model_name = "DeepMount00/mamba_790_hf_qa"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = MambaForCausalLM.from_pretrained(model_name, device_map={"": 0}).eval()
except OSError as e:
    print(f"Error loading model or tokenizer: {e}")
    exit()
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    exit()

def predict(contesto, domanda):
    device = "cuda:0"
    prefix_text = 'Di seguito ti verrà fornito un contesto e poi una domanda. il tuo compito è quello di rispondere alla domanda basandoti esclusivamente sul contesto\n\n'

    prompt = f"""{prefix_text}##CONTESTO: {contesto}\n##DOMANDA: {domanda}\n"""

    input_ids = tokenizer([prompt], return_tensors="pt").to(device)

    generate_ids = model.generate(**input_ids, max_new_tokens=150, eos_token_id=8112)

    answer = tokenizer.batch_decode(generate_ids)
    try:
        final_answer = answer[0].split("##RISPOSTA: ")[1].split("##END")[0].strip("\n")
    except IndexError:
        final_answer = ""
    return final_answer

context = """You are a law assistant, be helpful, answer longer as it require to help understand the law"""
question = """
        Explain the concept of "free and appropriate public education" (FAPE) under the Individuals with Disabilities Education Act (IDEA) in the United States. Discuss how FAPE ensures that students with disabilities receive an education tailored to their individual needs. Additionally, outline the key responsibilities of schools and parents in developing an Individualized Education Program (IEP) to comply with FAPE.

      Provide examples of cases or scenarios where schools may fail to meet FAPE requirements, and explain the potential legal remedies available to parents if their child's right to FAPE is denied.

      Conclude by discussing how the principles of FAPE can be balanced with budgetary constraints faced by schools.
      """

print(predict(context, question))

An unexpected error occurred: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 10.44 MiB is free. Including non-PyTorch memory, this process has 3.75 GiB memory in use. Of the allocated memory 3.69 GiB is allocated by PyTorch, and 310.00 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 2.44 MiB is free. Including non-PyTorch memory, this process has 3.75 GiB memory in use. Of the allocated memory 3.69 GiB is allocated by PyTorch, and 301.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

## Q-bert/Mamba-790M

In [ ]:
from transformers import AutoModelForCausalLM , AutoTokenizer

model = AutoModelForCausalLM.from_pretrained('Q-bert/Mamba-790M', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('Q-bert/Mamba-790M')

text = """
        Explain the concept of "free and appropriate public education" (FAPE) under the Individuals with Disabilities Education Act (IDEA) in the United States. Discuss how FAPE ensures that students with disabilities receive an education tailored to their individual needs. Additionally, outline the key responsibilities of schools and parents in developing an Individualized Education Program (IEP) to comply with FAPE.

      Provide examples of cases or scenarios where schools may fail to meet FAPE requirements, and explain the potential legal remedies available to parents if their child's right to FAPE is denied.

      Conclude by discussing how the principles of FAPE can be balanced with budgetary constraints faced by schools.

      """

input_ids = tokenizer.encode(text, return_tensors="pt")

output = model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


## state-spaces/mamba-790m-hf

In [ ]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-790m-hf")
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-790m-hf")
input_ids = tokenizer("Hey how are you doing?", return_tensors="pt")["input_ids"]

out = model.generate(input_ids, max_new_tokens=10)
print(tokenizer.batch_decode(out))
